In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 데이터 전처리

## 데이터 로드

In [2]:
from langchain.document_loaders import PDFPlumberLoader, TextLoader

import glob

# data 폴더 안의 모든 PDF 가져오기
docs = glob.glob("docling_outputs/*.md")

# docs = ["data/LangGraph-Based Integrated Architecture for CrewAI Multi-Agent.pdf",
#        "data/insight_Agentic.pdf"]

all_docs = []
for d in docs:
    loader = TextLoader(d, encoding="utf-8")
    docs = loader.load()
    all_docs.extend(docs)

In [3]:
print(all_docs)

[Document(metadata={'source': 'docling_outputs\\SPRi AI Brief 7월호 산업동향.md'}, page_content="<!-- image -->\n\nSPRi AI Brief\n\n2025년 7월호\n\n<!-- image -->\n\n## CONTENTS\n\n| 정책 ･ 법제   | ∙ OECD, AI와 인간의 능력을 비교한 AI 역량 지표 공개                      |   2 |\n|---------------|--------------------------------------------------------------------------|-----|\n|               | ∙ 일본 정부, 「AI 관련 기술의 연구개발 및 활용 추진에 관한 법률」 공포   |   3 |\n|               | ∙ 일본 방위성, AI를 활용한 무기의 연구개발 지침 발표                     |   4 |\n|               | ∙ 미국 상무부, AI안전연구소를 AI표준혁신센터로 개편                      |   5 |\n|               | ∙ EU 집행위원회 공동연구센터, 생성 AI 전망 보고서 발간                   |   6 |\n| 기업 ･ 산업   | ∙ 앤스로픽, 차세대 AI모델 '클로드 오푸스 4'와 '클로드 소네트 4'출시      |   8 |\n|               | ∙ 애플, WWDC 2025에서 '애플 인텔리전스'신기능 공개                       |   9 |\n|               | ∙ 미스트랄 AI, 추론 AI 모델과 기업용 AI 코딩 도구 출시                   |  10 |\n|               | ∙ AMD, 제품 발표 행사에서 개방형 AI 생태계 비전하에 신제품과 플랫폼 공개 |  11 |\n|               | ∙ 엔비디아

In [4]:
len(all_docs)

2

## 데이터 분할(Chunking)

### Semantic Chunking

In [ ]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain_ollama import OllamaEmbeddings

# bge_m3_embeddings = OllamaEmbeddings(model="bge-m3")
# semantic_chunker = SemanticChunker(bge_m3_embeddings)

# semantic_chunker_docs = semantic_chunker.split_documents(all_docs)
# len(semantic_chunker_docs)

21

### Recursive Character Text Splitter

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 일반 텍스트용 splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)
split_documents = text_splitter.split_documents(all_docs)

In [6]:
len(split_documents)

257

# SAVE: Qdant Vector Store

In [7]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, SparseVectorParams

In [9]:
# client = QdrantClient(":memory:") → 인메모리 벡터 저장소 생성.
client = QdrantClient(host="localhost", port=6333) # docker를 이용해 사용. 따라서 포트번호가 필요함.

collection_name = "RAG_Example(RAG_strategies)"

dense_embedding = OllamaEmbeddings(model="bge-m3")
sparse_embedding = FastEmbedSparse(model_name="Qdrant/bm25")

In [10]:
# Collection 생성
client.create_collection(
    collection_name=collection_name,
    vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
    sparse_vectors_config={"sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))},
)

True

In [11]:
qdrant = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=dense_embedding,
    sparse_embedding=sparse_embedding,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse"
)

qdrant.add_documents(split_documents)

['d9ce5573ff8a4aa6858d6ce9f8294ead',
 '59c2e2995b09416195c5ea1b89c9ebdc',
 'f91c1c08e2064e319b439845e79a8d1f',
 '496f8bc158ce4ae58c41177d9a060140',
 '8a7c75ef374243048787385972018b51',
 'a53cb90df9e742e2907b275d5e8c867c',
 'eaad31a2af6147929ec58b418dbf79f7',
 '384163de275646c7b65739b52abe2dda',
 'b129d6f80a0e4da9bf41bb4cf798a3be',
 '2e2905371f5e404f89fa2271a8015fd6',
 '37bae5ccb1a14387ae3aa0653d24279c',
 '2186ffbe44b745b080b349b0691731a8',
 'b84c482ec8c34e7586836d95e88bddb5',
 'af21c1f2af6c4d80b701efaebc4b4d3b',
 '7702de83678f40828c26c3b7cb715c7b',
 '05ea559d78974c7594be05ae34635eb7',
 '90c3083163aa4ff3b857553c4f2d68ae',
 'a3e549c018c8475cbb6ed2301761837c',
 'ce41142f3cf0468f8f6ed6303b85892d',
 '8232c23628bb4634bb8db785cee7e0b8',
 '14dded02c61e41e5a765105153e5d33e',
 '9692b210fd08435a88054c1fcda7ea1b',
 '4462b3aab36e4d1cbfaeb2ff5d849fb9',
 '0332989831d447599509eeecb5f85ddb',
 '18bad5271bfb4468a52156f5d84ec24e',
 'b5c43b198dbe442690632e4b8e89115b',
 'ebfb42f73cff4af4934e64d84f2652d8',
 

In [12]:
response = qdrant.as_retriever(search_kwargs={"k": 4})

In [13]:
query = "self-rag"
print(response.invoke(query)[0].page_content)

- (법적·정책적 요건) 국제인도법을 포함한 국제법과 국내법을 준수해야 하며, 인간의 개입 없이 AI가 자동으로 목표를 설정해 공격하는 자율살상무기(LAWS)의 연구개발을 금지
- (기술적 요건) AI 시스템 활용 시 인간의 참여와 통제를 가능하게 하고, 편향을 방지하고 검증 가능성과 투명성, 신뢰성을 보장하며, 오작동과 심각한 장애 발생 위험을 완화하는 체계를 마련해 안전성을 확보
- n (실시 사항) AI 무기의 연구개발 시 △AI 무기의 분류 △법적·정책적 심사 △기술적 심사의 3단계 심사를 거쳐 사업을 추진하며, 법적·정책적 심사와 기술적 심사는 고위험 무기에만 진행
- (AI 무기의 분류) 자율살상무기의 관점에서 집중적으로 위험을 관리해야 할 무기는 고위험으로, 그 외의 AI 무기는 저위험으로 분류하여 차등화된 관리를 실시*
* 저위험 AI 무기는 법적·정책적 심사와 기술적 심사를 면제하고 사업 수행 담당자의 자체 점검을 중심으로 위험관리를 진행


In [14]:
response.invoke(query)

[Document(metadata={'source': 'docling_outputs\\SPRi AI Brief 7월호 산업동향.md', '_id': '4462b3aa-b36e-4d1c-bfae-b2ff5d849fb9', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content='- (법적·정책적 요건) 국제인도법을 포함한 국제법과 국내법을 준수해야 하며, 인간의 개입 없이 AI가 자동으로 목표를 설정해 공격하는 자율살상무기(LAWS)의 연구개발을 금지\n- (기술적 요건) AI 시스템 활용 시 인간의 참여와 통제를 가능하게 하고, 편향을 방지하고 검증 가능성과 투명성, 신뢰성을 보장하며, 오작동과 심각한 장애 발생 위험을 완화하는 체계를 마련해 안전성을 확보\n- n (실시 사항) AI 무기의 연구개발 시 △AI 무기의 분류 △법적·정책적 심사 △기술적 심사의 3단계 심사를 거쳐 사업을 추진하며, 법적·정책적 심사와 기술적 심사는 고위험 무기에만 진행\n- (AI 무기의 분류) 자율살상무기의 관점에서 집중적으로 위험을 관리해야 할 무기는 고위험으로, 그 외의 AI 무기는 저위험으로 분류하여 차등화된 관리를 실시*\n* 저위험 AI 무기는 법적·정책적 심사와 기술적 심사를 면제하고 사업 수행 담당자의 자체 점검을 중심으로 위험관리를 진행'),
 Document(metadata={'source': 'docling_outputs\\SPRi AI Brief 7월호 산업동향.md', '_id': '4f5c6f30-4a5a-4486-97ef-5a2d5d94f86e', '_collection_name': 'RAG_Example(RAG_strategies)'}, page_content='## £ 마지스트랄, 여타 추론 AI 모델보다 최대 10배 빠른 속도로 답변을 제공'),
 Document(metadata={'source': 'docling_outputs\\SPRi AI Brief 7월호 산업동향.md', '_id